In [1]:
%load_ext autoreload
%autoreload 2

%env PUBLIC_BACKEND_ENDPOINT=http://localhost:8000

from zeno_api import ZenoClient

import pandas as pd
import ast
import json
from pathlib import Path

import pandas as pd
import zeno_api

env: PUBLIC_BACKEND_ENDPOINT=http://localhost:8000


In [12]:
USERNAME = "test"
PASSWORD = "Test12345!"
ENDPOINT = "http://localhost:8000"
PROJECT_NAME = "test_project"
PROJECT_VIEW = "text-classification"
EXISTING_PROJECT_PATH = "test.csv"

In [7]:
zeno_client = ZenoClient(
    username=USERNAME, password=PASSWORD, endpoint=ENDPOINT
)

In [8]:
project = zeno_client.create_project(
    PROJECT_NAME, 
    view=PROJECT_VIEW
)

Exception: Not Found

In [ ]:

data_frame = pd.read_csv(EXISTING_PROJECT_PATH)
data_frame = data_frame.fillna("")
output_cols = list(
    filter(lambda col: str(col).startswith("OUTPUToutput"), data_frame.columns)
)
models = list(
    set(list(map(lambda col: str(col).replace("OUTPUToutput", ""), output_cols)))
)
predistill_cols = list(
    filter(lambda col: str(col).startswith("PREDISTILL"), data_frame.columns)
)
postdistill_cols = list(
    filter(lambda col: str(col).startswith("POSTDISTILL"), data_frame.columns)
)

Uploading existing zeno project to a zeno backend using the Zeno API.

We assume that this existing project has been created with an older version of zeno.
Therefore, this script makes assumptions about the column names and data structure of
the CSV.